<a href="https://colab.research.google.com/github/hager2164/DL_tasks/blob/main/VGG11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
ngpu = 2
device =torch.device("cuda:0" if (torch.cuda.is_available()and ngpu >0 ) else "cpu")

In [3]:
!nvidia-smi


Fri Sep 19 14:51:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
def data_loader(data_dir , batch_size , random_seed = 42  , valid_size = 0.1 , shuffle=True ,test = False ):
    normalize = transforms.Normalize(    mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],)
    transform = transforms.Compose([transforms.Resize((227,227)),
                                    transforms.ToTensor(),
                                    normalize ,])
    if test :
        dataset = datasets.CIFAR100(
            root=data_dir , train= False ,
            download = True , transform = transform,
        )
        data_loader= torch.utils.data.DataLoader(
            dataset , batch_size = batch_size , shuffle = shuffle
        )
        return data_loader
    train_dataset = datasets.CIFAR100(
        root=data_dir , train= True ,
        download = True , transform = transform,
        )
    valid_dataset = datasets.CIFAR100(
        root=data_dir , train= True ,
        download = True , transform = transform,
        )
    num_train= len(train_dataset)
    indices = list(range(num_train))
    split = int (np.floor(valid_size*num_train))
    if shuffle :
        np.random.seed(random_seed)
        np.random.shuffle(indices)


    train_idx , valid_idx = indices[split:] , indices[split:]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)


    train_loader = torch.utils.data.DataLoader(train_dataset , batch_size= batch_size  , sampler = train_sampler)
    valid_loader = torch.utils.data.DataLoader(valid_dataset , batch_size= batch_size  , sampler =valid_sampler)
    return (train_loader , valid_loader)

In [5]:
train_loader ,valid_loader = data_loader(data_dir= './data' , batch_size = 64)
test_loader = data_loader(data_dir= './data' , batch_size = 64, test = True )

100%|██████████| 169M/169M [00:05<00:00, 33.1MB/s]


In [6]:
class VGG11(nn.Module):
  def __init__(self ,ngpu , num_classes=10 ):
    super(VGG11 ,self).__init__()
    self.ngpu = ngpu = ngpu
    self.layer1 = nn.Sequential(
        nn.Conv2d(3,64 ,kernel_size=3 ,stride =1  ,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2 , stride = 2 ))

    self.layer2 = nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3 ,stride =1  ,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2 , stride = 2 ))

    self.layer3 = nn.Sequential(
        nn.Conv2d(128,256 ,kernel_size=3 ,stride =1  ,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2 , stride = 2 ))

    self.layer4= nn.Sequential(
        nn.Conv2d(256,256 ,kernel_size=3 ,stride =1  ,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2 , stride = 2 ))

    self.layer5= nn.Sequential(
        nn.Conv2d(256,512 ,kernel_size=3 ,stride =1  ,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2 , stride = 2 ))

    self.layer6 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

    self.layer7 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())

    self.layer8 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

    self.fc = nn.Sequential(nn.Dropout(0.5),
            nn.Linear(512, 4096),
               nn.ReLU())
    self.fc1 = nn.Sequential(nn.Dropout(0.5),
            nn.Linear(4096, 4096),
               nn.ReLU())
    self.fc2  = nn.Sequential(nn.Linear(4096, num_classes))


  def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
num_classes = 100
num_epochs = 25
batch_size = 32
learning_rate = 0.005

model = VGG11(ngpu , num_classes).to(device)

In [8]:
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

0 Tesla T4


In [9]:

if (device.type =='cuda') and (ngpu > 1):
  model = nn.DataParallel(model , list(range(ngpu)))

AssertionError: Invalid device id

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate , weight_decay=0.005 , momentum=0.9 )


In [11]:
total = len (train_loader)

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total, loss.item()))

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/25], Step [704/704], Loss: 3.7467
Accuracy of the network on the 5000 validation images: 15.053333333333333 %
Epoch [2/25], Step [704/45000], Loss: 3.4820
Accuracy of the network on the 5000 validation images: 24.697777777777777 %
Epoch [3/25], Step [704/45000], Loss: 2.3880
Accuracy of the network on the 5000 validation images: 33.07333333333333 %
Epoch [4/25], Step [704/45000], Loss: 2.6477
Accuracy of the network on the 5000 validation images: 39.64888888888889 %
Epoch [5/25], Step [704/45000], Loss: 1.9259
Accuracy of the network on the 5000 validation images: 44.79333333333334 %
Epoch [6/25], Step [704/45000], Loss: 2.4684
Accuracy of the network on the 5000 validation images: 49.4 %
Epoch [7/25], Step [704/45000], Loss: 2.8411
Accuracy of the network on the 5000 validation images: 54.9 %
Epoch [8/25], Step [704/45000], Loss: 2.4850


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))